In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Strawberry___healthy', 'Strawberry___Leaf_scorch']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(42585, 128, 128, 3)

In [8]:
y.shape

(42585, 23)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(34068, 128, 128, 3)

In [11]:
X_test.shape

(8517, 128, 128, 3)

In [12]:
y_train.shape

(34068, 23)

In [13]:
y_test.shape

(8517, 23)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

23

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 22:45:48.993085: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1065/1065 [==============================] - 207s 194ms/step - loss: 1.0651 - accuracy: 0.6659 - val_loss: 1.5155 - val_accuracy: 0.5948
Epoch 2/10
1065/1065 [==============================] - 221s 208ms/step - loss: 0.4399 - accuracy: 0.8551 - val_loss: 0.3174 - val_accuracy: 0.8895
Epoch 3/10
1065/1065 [==============================] - 222s 209ms/step - loss: 0.2941 - accuracy: 0.9027 - val_loss: 0.2512 - val_accuracy: 0.9150
Epoch 4/10
1065/1065 [==============================] - 223s 209ms/step - loss: 0.2244 - accuracy: 0.9245 - val_loss: 0.1878 - val_accuracy: 0.9344
Epoch 5/10
1065/1065 [==============================] - 223s 210ms/step - loss: 0.1843 - accuracy: 0.9386 - val_loss: 0.1200 - val_accuracy: 0.9601
Epoch 6/10
1065/1065 [==============================] - 222s 208ms/step - loss: 0.1608 - accuracy: 0.9459 - val_loss: 0.1429 - val_accuracy: 0.9539
Epoch 7/10
1065/1065 [==============================] - 221s 208ms/step - loss: 0.1306 - accuracy: 0.9574 - val_loss: 0.424

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 96.17%


In [19]:
cnn_scores

[0.10505586117506027, 0.9617236256599426]

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

267/267 [==============================] - 11s 41ms/step
Accuracy: 96.17%
Precision: 96.50%
Recall: 96.17%
F1 score: 96.17%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-8-jenis-daun.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | afcad9ca-9bd2-4c7b-b034-2b3d0360ac30___RS_LB 2741.JPG                         | Potato___Late_blight                               | Potato___Late_blight                               |
|    1 | 6b5b0495-3fa5-4da0-894b-3a0ec775a0fd___RS_L.Scorch 1233_flipLR.JPG            | Strawberry___Leaf_scorch                           | Strawberry___Leaf_scorch                           |
|    2 | 963ec7cb-0269-44

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-8-jenis-daun.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                 | True Labels                                        | Predicted Labels                                   |
|------+--------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    7 | 599ee05a-0659-4159-a65f-43278e6f8f55___RS_NLB 3841.JPG                   | Corn_(maize)___Northern_Leaf_Blight                | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot |
|    8 | 66fb9e0e-39cb-4a5f-8780-1bcfbda5577c___RS_NLB 3919_180deg.JPG            | Corn_(maize)___Northern_Leaf_Blight                | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot |
|   76 | 1bc60205-27cc-4d10-bc8

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

1065/1065 [==============================] - 62s 58ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 8897.57548880577 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

267/267 [==============================] - 41s 154ms/step
Random Forest Classifier Accuracy: 97.95%
Random Forest Classifier Precision: 0.98
Random Forest Classifier Recall: 0.98
Random Forest Classifier F1 Score: 0.98


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-8-jenis-daun.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | afcad9ca-9bd2-4c7b-b034-2b3d0360ac30___RS_LB 2741.JPG                         | Potato___Late_blight                               | Potato___Late_blight                               |
|    1 | 6b5b0495-3fa5-4da0-894b-3a0ec775a0fd___RS_L.Scorch 1233_flipLR.JPG            | Strawberry___Leaf_scorch                           | Strawberry___Leaf_scorch                           |
|    2 | 963ec7cb-0269-44

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-8-jenis-daun.csv', index=False)

False Predictions:
+------+----------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                   | True Labels                                        | Predicted Labels                                   |
|------+----------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   76 | 1bc60205-27cc-4d10-bc87-7e968573e5d5___FREC_Scab 3531_270deg.JPG           | Apple___Apple_scab                                 | Apple___Black_rot                                  |
|   98 | f73f0a24-d25e-4a0c-9e68-3b57d467090d___JR_B.Spot 8829.JPG                  | Pepper,_bell___Bacterial_spot                      | Apple___Black_rot                                  |
|  156 | e1e8a707-d2b